In [1]:
import pandas as pd

## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [101]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
agg_ratings = (ratings[['userId', 'timestamp']]
               .groupby('userId')
               .agg(['count', 'min', 'max'])
               [(agg_ratings[('timestamp', 'count')] > 100)]
              )
agg_ratings


timestamp                        
           count         min         max
userId                                  
4            204   949778714   949982274
8            116  1154389340  1154474527
15          1700   997937239  1469330735
17           363  1127468587  1127476640
19           423   855190091   855195373
...          ...         ...         ...
656          128   986240991   986244044
659          142   834598040   866207451
664          519  1343731283  1441911722
665          434   992836298  1046967769
671          115  1063500751  1074784735

[258 rows x 3 columns]

In [104]:
agg_ratings['life_time'] = agg_ratings.apply(lambda x: x[('timestamp',   'max')] - x[('timestamp',   'min')], axis=1)
agg_ratings['life_time'].mean()

40080507.4496124

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [2]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [3]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [4]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)


,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [63]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [42]:
rzd['type'] = 'rzd'
auto['type'] = 'auto'
air['type'] = 'air'

rzd.rename(columns={'rzd_revenue': 'revenue'}, inplace=True)
auto.rename(columns={'auto_revenue': 'revenue'}, inplace=True)
air.rename(columns={'air_revenue': 'revenue'}, inplace=True)


revenue_df = rzd.append(auto).append(air)
revenue_df

,client_id,revenue,type
0,111,1093,rzd
1,112,2810,rzd
2,113,10283,rzd
3,114,5774,rzd
4,115,981,rzd
0,113,57483,auto
1,114,83,auto
2,115,912,auto
3,116,4834,auto
4,117,98,auto


In [66]:
pivot_df = revenue_df.pivot_table(index='client_id', values='revenue', columns='type', aggfunc='sum', fill_value=0)
pivot_df

type,air,auto,rzd
client_id,,,
111,0,0,1093
112,0,0,2810
113,0,57483,10283
114,0,83,5774
115,81,912,981
116,4,4834,0
117,13,98,0
118,173,0,0


In [70]:
client_base.join(pivot_df, on='client_id')

,client_id,address,air,auto,rzd
0,111,Комсомольская 4,0,0,1093
1,112,Энтузиастов 8а,0,0,2810
2,113,Левобережная 1а,0,57483,10283
3,114,Мира 14,0,83,5774
4,115,ЗЖБИиДК 1,81,912,981
5,116,Строителей 18,4,4834,0
6,117,Панфиловская 33,13,98,0
7,118,Мастеркова 4,173,0,0


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1) Нужно использовать время получения координат и время визита/покупки для привязки координат к действиям пользователя. Условно для каждого визита вычисляем интервал [время визита - 2 секунды, время визита + 2 секунды]. Дальше для каждого пользователя для каждого лога координат ищем вхождение по времени.

2) Видимо я не совсем понял суть вопроса. Не вижу проблему в разных координатах. Мы привязываем по пользователю и времени и разница в координатах не должна отразиться на формировании общей таблицы

3) Из координат можно предположить, где пользователь находился в момент времени. Через карты можем вытащить адреса, районы и типы зданий (бц, жилой дом и т.д.). + в связке со временем можно предположить, где пользователь предпочитает выбирать товары, а где - приобретать. Условно если было 2 визита днем в районе таганки и покупка вечером в жилом доме, значит в следующий раз стоит показывать рекламу с новыми товарами в рабочее время, а вечером кидать офер на просмотренные позиции. 